In [ ]:
import numpy as np
import pandas as pd

In [ ]:
rho_tank=7800 #kg/m3
m=0.0007777777777777777#ton/s
P=410#bar
T=273.15+25
T1=T
Tref=10+273.15
Po=31
# Coeficientes del O2
coeff_O2 = [29.79023995, -0.009488538, 2.85799E-05, 9.87286E-09, -5.66511E-11, 4.30016E-14, -1.02189E-17]
# Función de entalpía específica del O2
def enthalpy_O2(T,T_ref):
    return (sum(coeff_O2[n] / (n + 1) * (T**(n + 1) - T_ref**(n + 1)) for n in range(7)))/32

In [ ]:
import pandas as pd
crit=pd.DataFrame()

crit['Comp']=['O2']
crit['Tc']=[154.58]
crit['Pc']=[4985.19]
crit['w']=[0.0250]
crit['PM']=[32]

"""
crit['Comp']=['H2']
crit['Tc']=[32.938]
crit['Pc']=[1293.00]
crit['w']=[-0.2160]
crit['PM']=[1.00784*2]
"""


T=298
#P1=100
def PRv2(Tc,Pc,mm,w,T,P):
    #Unidades: P, Pc: kPa, T, Tc: K, w: Adim, mm: kg/kmol
    R=8.314#J/mol-K
    Ru=R/mm
    mi=0.379642+1.48503*w-0.164423*w**2+0.016666*w**3
    alpha=(1+mi*(1-(T/Tc)**0.5))**2
    a=0.45724*((Ru)**2)*(Tc**2)*alpha/(Pc)
    b=0.0778*(Ru)*(Tc/Pc)
    A_may=a*P/(((Ru)**2)*(T**2))
    B_may=b*P/((Ru)*T)
    m=-(1-B_may)
    n=(A_may-(3*B_may**2)-(2*B_may))
    o=-((A_may*B_may)-(B_may**2)-(B_may**3))
    v=(Ru*T)/P
    return {'alpha':alpha,'a':a,'b':b,'A':A_may,'B':B_may,'m':m,'n':n,'o':o,'v':v}
resul_peng=PRv2(crit['Tc'][0],crit['Pc'][0],crit['PM'][0],crit['w'][0],316+273.15,172.37*100)
z=np.real(max(np.roots([1,resul_peng['m'],resul_peng['n'],resul_peng['o']])))
#r1=(P1*crit['PM'][0])/(z*(8.314)*(T+273.15))
#V1=m/r1
#V1

In [ ]:
import numpy as np
from scipy.integrate import quad
from scipy.optimize import fsolve
import scipy.optimize as opt

def find_required_area(demand, irradiance_values, PV_eff=0.1):
    """
    Encuentra el área A (en m²) requerida para que la potencia efectiva de la planta PV
    sea igual a la demanda.
    
    Parámetros:
      demand             : Demanda en kW (por ejemplo, 50*1.25)
      irradiance_values  : Lista o array de valores de irradiancia en W/m².
      PV_eff             : Eficiencia del sistema fotovoltaico (por defecto 0.1 = 10%)
    
    Retorna:
      A_required         : Área requerida en m², tal que:
                           (Promedio(irradiancia)*A_required/1000)*PV_eff = demand.
    """
    # Calcular la irradiancia promedio (W/m²)
    Is_mean = np.mean(irradiance_values)
    
    # Función objetivo: (Is_mean * A / 1000) * PV_eff debe ser igual a demand
    def objective(A):
        production = (Is_mean * A[0] / 1000) * PV_eff  # en kW
        print(f"Producción: {production} kW, Demanda: {demand} kW")
        return production - demand

    # Valor inicial (por ejemplo, 1000 m²)
    A_initial = 10000
    A_solution = opt.root(objective, A_initial, 
                           method='hybr',tol=1e-2, options={'maxfev': 1000000})

    if A_solution.success:
        return A_solution.x[0]
    else:
        raise RuntimeError("No se encontró la solución: " + A_solution.message)

def compute_pv_system_analysis(demand, irradiance_values, PV_eff=0.1, time_hours=4):
    """
    Realiza el análisis completo de un sistema PV: optimiza el área requerida y calcula
    diversas energías (producción, integración, acoplamiento entre generación y demanda).
    
    Parámetros:
      demand             : Demanda en kW (ya ajustada, por ejemplo, demand = 50*1.25)
      irradiance_values  : Lista o array de valores de irradiancia en W/m².
      PV_eff             : Eficiencia del sistema PV (por defecto 0.1, es decir, 10%).
      time_hours         : Tiempo de operación en horas para el que se calcula la energía.
      
    Retorna:
      results (dict)     : Diccionario con los siguientes valores:
           - "Area_required_m2"       : Área requerida (m²).
           - "demand_kW"              : Demanda en kW.
           - "irradiance_mean_W_m2"   : Irradiancia media en W/m².
           - "Ps_inst_kW"             : Potencia solar instantánea teórica en kW.
           - "Pv_kW"                  : Potencia efectiva PV en kW.
           - "E_pv_constant_kWh"      : Energía producida suponiendo potencia constante (kWh).
           - "E_demand_integrated_kWh": Energía demandada integrada (kWh).
           - "err_demand"             : Error estimado en la integración de la demanda.
           - "E_pv_integrated_kWh"    : Energía producida (integración con potencia constante) (kWh).
           - "err_PV"                 : Error estimado en la integración de la PV (constante).
           - "E_pv_linear_kWh"        : Energía producida considerando un perfil lineal (kWh).
           - "err_PV_linear"          : Error estimado en la integración del perfil lineal.
           - "Esupplied_kWh"          : Energía suministrada efectivamente (kWh).
           - "Eunsupplied_kWh"        : Energía no suministrada (kWh).
           - "Eunused_kWh"            : Energía solar no utilizada (kWh).
    """
    # Optimización: encontrar el área requerida para que la producción PV iguale a la demanda
    A_required = find_required_area(demand, irradiance_values, PV_eff)

    # Cálculo de parámetros de irradiancia y potencia
    Is_mean = np.mean(irradiance_values)  # en W/m²
    Ps_inst = (Is_mean * A_required) / 1000  # Potencia solar instantánea en kW
    Pv = Ps_inst * PV_eff  # Potencia efectiva de la planta PV en kW

    # Energía eléctrica producida durante el período (potencia constante)
    E_pv = Pv * time_hours  # en kWh

    # =============================================================================
    # Cálculo del acoplamiento entre generación y demanda
    # =============================================================================
    if Pv > demand:
        Esup = demand * time_hours   # La planta puede cubrir toda la demanda
    else:
        Esup = Pv * time_hours        # Solo se suministra lo que produce la planta

    if Pv < demand:
        Eunsup = (demand - Pv) * time_hours  # Energía no suministrada
    else:
        Eunsup = 0

    if Pv > demand:
        Eunused = (Pv - demand) * time_hours  # Energía solar no utilizada
    else:
        Eunused = 0


    price = 755
    price = price * (293.9/239.1)
    SPP = price * Ps_inst
    cost = SPP * 1.35
    # Compilamos los resultados en un diccionario
    results = {
        "Area_required_m2": A_required,
        "demand_kW": demand,
        "irradiance_mean_W_m2": Is_mean,
        "Ps_inst_kW": Ps_inst,
        "Pv_kW": Pv,
        "E_pv_constant_kWh": E_pv,
        "Esupplied_kWh": Esup,
        "Eunsupplied_kWh": Eunsup,
        "Eunused_kWh": Eunused,
        "Cost": cost
    }
    
    return results

In [ ]:
import numpy as np
from scipy.optimize import root
def trabajo_reversible(m, P1, P2, eta, T2,T1):
    P1=P1*100
    P2=P2*100
    
    resul_peng=PRv2(crit['Tc'][0],crit['Pc'][0],crit['PM'][0],crit['w'][0],T1,P1)
    z=np.real(max(np.roots([1,resul_peng['m'],resul_peng['n'],resul_peng['o']])))
    r1=(P1*crit['PM'][0])/(z*(8.314)*T1)
    
    V1=1/r1
    resul_peng=PRv2(crit['Tc'][0],crit['Pc'][0],crit['PM'][0],crit['w'][0],T2+273.15,P2)
    z=np.real(max(np.roots([1,resul_peng['m'],resul_peng['n'],resul_peng['o']])))
    r2=(P2*crit['PM'][0])/(z*(8.314)*(T2+273.15))
    V2=1/r2
    n = np.log(P2/P1)/(np.log((V1)/(V2)))
    
    term1 = (m * P1 * 1000 * V1) / (0.72*(n - 1)/n)
    term2 = (P2 / P1) ** ((n - 1) / n) - 1
    W_rev=term1*term2/1000
    return W_rev/eta

def trabajo_reversible_IT(tt, m, P1, P2, eta,T1):
    P1=P1*100
    P2=P2*100
    T2=abs(tt[0])

    resul_peng=PRv2(crit['Tc'][0],crit['Pc'][0],crit['PM'][0],crit['w'][0],T1,P1)
    z=np.real(max(np.roots([1,resul_peng['m'],resul_peng['n'],resul_peng['o']])))
    r1=(P1*crit['PM'][0])/(z*(8.314)*T1)

    V1=1/r1
    resul_peng=PRv2(crit['Tc'][0],crit['Pc'][0],crit['PM'][0],crit['w'][0],T2,P2)
    z=np.real(max(np.roots([1,resul_peng['m'],resul_peng['n'],resul_peng['o']])))
    r2=(P2*crit['PM'][0])/(z*(8.314)*(T2))
    V2=1/r2
    n = np.log(P2/P1)/(np.log((V1)/(V2)))
    
    term1 = (1 * P1 * V1 * 1000) / (0.72*(n - 1)/n)
    term2 = (P2 / P1) ** ((n - 1) / n) - 1
    Tref=T1
    T=T2
    HhH2=(19.67099783*(T-Tref) + 0.069681519/2*(T**2-Tref**2) - 0.000200098/3*(T**3-Tref**3) + 2.89493E-07/4*(T**4-Tref*4) - 2.22475E-10/5*(T**5-Tref**5) + 8.81466E-14/6*(T**6-Tref**6) - 1.42043E-17/7*(T**7-Tref**7))/2
    HhO2=enthalpy_O2(T,Tref)
    T=T1
    HhH1=(19.67099783*(T-Tref) + 0.069681519/2*(T**2-Tref**2) - 0.000200098/3*(T**3-Tref**3) + 2.89493E-07/4*(T**4-Tref*4) - 2.22475E-10/5*(T**5-Tref**5) + 8.81466E-14/6*(T**6-Tref**6) - 1.42043E-17/7*(T**7-Tref**7))/2
    HhO1=enthalpy_O2(T,Tref)
    W_rev=term1*term2/1000
    print('kW',W_rev)
    return W_rev - (HhO2 - HhO1)

P1=31
P2=141.82
m=m
eta=0.75
sol = root(trabajo_reversible_IT, x0=500, args=(m, P1, P2, eta,298.15), method='hybr')
if sol.success:
    print(sol.message)
else:
    print(sol.message)

resultado = trabajo_reversible(m, P1, P2, eta, sol.x[0]-273.15,298.15)
print(f"W: {resultado:.20f} kW")
sol.x-273.15

In [ ]:
m*86400

In [ ]:
Tref

In [ ]:
import numpy as np
from scipy.optimize import root

def balance(x, T, Tref, m):
    global e
    global s
    """
    Calcula la función de balance para el proceso,
    devolviendo la diferencia e - s.

    Parámetros:
      x    : vector de incógnitas; se asume que x[0] es la variable a encontrar.
      T    : temperatura en K.
      Tref : temperatura de referencia en K.
      m    : masa del hidrógeno (kg).

    La función utiliza dos series polinómicas (para H2 y H2O)
    para calcular, respectivamente, las energías 'e' y 's', y retorna e - s.
    """
    # Calcular la entalpía (u otra propiedad) para hidrógeno
    Ta=15+273.15
    Ts=92+273.15
    HhH2 = ( 19.67099783*(T - Tref)
              + (0.069681519/2) * (T**2 - Tref**2)
              - (0.000200098/3) * (T**3 - Tref**3)
              + (2.89493E-07/4) * (T**4 - Tref**4)
              - (2.22475E-10/5) * (T**5 - Tref**5)
              + (8.81466E-14/6) * (T**6 - Tref**6)
              - (1.42043E-17/7) * (T**7 - Tref**7)
            ) / 2
    
    HhH2=enthalpy_O2(T,Tref)


    # Calcular la entalpía (u otra propiedad) para agua (H2O)
    HhH2O = ( -22.41701677*(Ta - Tref)
               + (0.876972156/2) * (Ta**2 - Tref**2)
               - (0.002570393/3) * (Ta**3 - Tref**3)
               + (2.48383E-06/4) * (Ta**4 - Tref**4)
             ) / 18.015
    
    
    
    Tk=25+273.15
    HhH22 = ( 19.67099783*(Tk - Tref)
              + (0.069681519/2) * (Tk**2 - Tref**2)
              - (0.000200098/3) * (Tk**3 - Tref**3)
              + (2.89493E-07/4) * (Tk**4 - Tref**4)
              - (2.22475E-10/5) * (Tk**5 - Tref**5)
              + (8.81466E-14/6) * (Tk**6 - Tref**6)
              - (1.42043E-17/7) * (Tk**7 - Tref**7)
            ) / 2
    HhH22=enthalpy_O2(Tk,Tref)

    # Calcular la entalpía (u otra propiedad) para agua (H2O)
    HhH2O2 = ( -22.41701677*(Ts - Tref)
               + (0.876972156/2) * (Ts**2 - Tref**2)
               - (0.002570393/3) * (Ts**3 - Tref**3)
               + (2.48383E-06/4) * (Ts**4 - Tref**4)
             ) / 18.015
    m=m*1e3
    s = (HhH22 + HhH2O2)* x[0]
    e = (HhH2 + HhH2O) * m
    
    return e - s


In [ ]:

import pandas as pd
import math

class PriceCalculator:
    def __init__(self, dataframe,spares,material):
        self.dataframe = dataframe
        self.spares = spares
        self.material = material
        self.calculate_price()

    def calculate_price(self):
        prices = []
        cepci_2001 = 397
        cepci_2024 = 798.8
        k1=2.2897
        k2=1.3604
        k3=-0.1027
        if self.material == "Carbon Steel":
            sFBM=3.4
        elif self.material == "Stainless Steel":
            sFBM=7.0
        else:
            sFBM=13.9 #for Nickel Alloy
        FBMCS=3.4 #for Carbon Steel

        for index, row in self.dataframe.iterrows():
            if index < len(self.dataframe) - 1:  # For all rows except the last one
                w_value = row["W (kW)"]
                A = k1+(k2*math.log10(w_value))+(k3*(math.log10(w_value)**2))
                B=(10**A)*(self.spares+1)/cepci_2001*cepci_2024
                price=B*sFBM/FBMCS
                prices.append(price)
            else:  # For the last row
                total_price = sum(prices)
                prices.append(total_price)
        self.dataframe["Purchased Equipment Cost (USD)"] = prices
        #self.dataframe["Purchased Equipment Cost (USD)"] = self.dataframe["Purchased Equipment Cost (USD)"].apply(lambda x: f"${x:,.2f}" if isinstance(x, (int, float)) else x)
        self.total_price = prices[-1]  # Total price is the last value in the list
        installation_cost = total_price * 2.5
        piping_cost = self.total_price * 0.21
        electrical_cost = self.total_price * 0.16
        instrumentation_cost = self.total_price * 0.08
        Final_cost = self.total_price + piping_cost + electrical_cost + instrumentation_cost+installation_cost
        cost_mxn= Final_cost * 19.59 
        pricce_mxn=self.total_price*19.59


        # Create a data frame with the extra costs
        extras = {
            "Purchased Equipment Cost (USD)": [self.total_price],
            "Purchased Equipment Cost (MXN)": [pricce_mxn],
            "Installation Cost (USD)": [installation_cost],
            "Piping Cost (USD)": [piping_cost],
            "Electrical Cost (USD)": [electrical_cost],
            "Instrumentation Cost (USD)": [instrumentation_cost],
            "Final Cost (USD)": [Final_cost],
            "Final Cost (MXN)": [cost_mxn]
            
        }
        self.extras_df = pd.DataFrame(extras)
        # Format the numbers in the extras_df as currency
        self.extras_df = self.extras_df.applymap(lambda x: f"${x:,.2f}" if isinstance(x, (int, float)) else x)
        self.final=Final_cost
        return self.dataframe, self.extras_df

# Use the class with the prueba dataframe
SS="Stainless Steel"
CS="Carbon Steel"
NA="Nickel Alloy"



In [ ]:
import numpy as np
from scipy.optimize import root

def calcular_dimension_tanque(P, m, crit, tipo, rho_tank, T=300, x0=[1, 4]):
    """
    Calcula las dimensiones, espesor y masa de un tanque en base a parámetros de entrada.
    
    Parámetros:
      - P: Presión de entrada (se espera en unidades que requieran ser convertidas a Pascales).
      - m: Masa (kg) o cantidad similar requerida para el cálculo del volumen.
      - crit: Diccionario con las propiedades críticas, debe incluir las claves:
              'Tc', 'Pc', 'PM' y 'w'. Se usan los primeros elementos (ej: crit['Tc'][0]).
      - tipo: Tipo de tanque, str. Debe ser 'hemi' o 'semi'.
      - rho_tank: Densidad del material del tanque (kg/m³).
      - T: Temperatura en Kelvin (por defecto 300 K).
      - x0: Valor inicial para la solución del sistema [r, L] (por defecto [1.5, 3.0]).
      
    Retorna:
      Un diccionario con los siguientes valores:
        - 'volumen_total': Volumen calculado del tanque (m³).
        - 'radio': Radio del tanque (m).
        - 'longitud': Longitud del tanque (m).
        - 'espesor': Espesor del tanque (m).
        - 'masa': Masa del tanque (ton).
        - 'V_objetivo': Volumen objetivo calculado (m³).
    
    Se lanzará un ValueError en caso de que el sistema no encuentre solución o 
    si el tipo de tanque no es reconocido.
    """
    
    # Se recalcula V_objetivo usando el método de Peng–Robinson (se asume PRv2 está definida)
    resul_peng = PRv2(crit['Tc'][0], crit['Pc'][0], crit['PM'][0], crit['w'][0], T, (P)*100)
    z = np.real(max(np.roots([1, resul_peng['m'], resul_peng['n'], resul_peng['o']])))
    r1 = (100 * P * crit['PM'][0]) / (z * 8.314 * T)
    V1 = m / r1
    V_objetivo = V1

    # Definición del sistema de ecuaciones para resolver por el método root
    def ecuaciones(x, V_obj):
        r, L = abs(x[0]), abs(x[1])
        if tipo == 'hemi':
            V_tank = (4 / 3) * np.pi * r**3 + (L - 2*r) * np.pi * r**2
        elif tipo == 'semi':
            V_tank = np.pi * r**2 * L + (2/3) * np.pi * r**3
        else:
            raise ValueError("Tipo de tanque no reconocido. Usa 'hemi' o 'semi'.")
        
        f1 = V_tank - V_obj  # La diferencia entre el volumen calculado y el objetivo
        f2 = L - 4 * r     # Relación de diseño entre L y r
        return [f1, f2]

    # Resolver el sistema de ecuaciones
    solucion = root(
    ecuaciones, 
    x0=x0, 
    args=(V_objetivo,), 
    method='hybr', 
    tol=1e-3, 
    options={'maxfev': 10000000}  # limite máximo de evaluaciones (iteraciones)
)

    if not solucion.success:
        raise ValueError("No se encontró solución: " + solucion.message)
    
    r, L_encontrado = solucion.x[0], solucion.x[1]

    # Calcular el volumen total del tanque según el tipo
    if tipo == 'hemi':
        V_total_sol = (4 / 3) * np.pi * r**3 + (L_encontrado - 2*r) * np.pi * r**2
    elif tipo == 'semi':
        V_total_sol = (2 / 3) * np.pi * r**3 + (L_encontrado - r) * np.pi * r**2
    else:
        raise ValueError("Tipo de tanque no reconocido. Usa 'hemi' o 'semi'.")

    # Cálculo del espesor del tanque usando una ecuación de pared delgada
    FS = 1.5  # Factor de seguridad
    Sy = 2800 * 100000
    Su = 3095 * 100000
    t1 = (100000*P * r * FS) / Sy
    t2 = (100000*P * r * 2.25) / Su
    t = max(t1, t1)  # Espesor del tanque (en metros)

    # Para el cálculo de la masa se asume h = r
    h = r
    if tipo == 'hemi':
        mt = rho_tank * ((4/3) * np.pi * (r + t)**3 + np.pi * (r + t)**2 * L_encontrado - V_objetivo)
    elif tipo == 'semi':
        mt = rho_tank * (np.pi * ((r + t)**2 * L_encontrado + (2/3) * (r + t)**2 * (h + t)) -
                         (np.pi * r**2 * L_encontrado + (2/3) * np.pi * r**2 * h))
    else:
        raise ValueError("Tipo de tanque no reconocido. Usa 'hemi' o 'semi'.")
    mt = mt / 1000  # Conversión a toneladas

    # Impresión de resultados
    print("Volumen calculado del tanque:", round(V_total_sol, 2), "m³")
    print("Radio del tanque:", round(r, 2), "m")
    print("Longitud del tanque:", round(L_encontrado, 2), "m")
    print("Espesor del tanque:", round(t*1000, 2), "mm")
    print("Masa del tanque:", round(mt, 2), "ton")

    # Retorna los resultados en un diccionario
    return {
        'volumen_total': V_total_sol,
        'radio': r,
        'longitud': L_encontrado,
        'espesor': t,
        'masa': mt,
        'V_objetivo': V_objetivo,
        'h2':m
    }

# Ejemplo de uso:
# Se debe definir o importar la función PRv2 y el diccionario crit apropiadamente.
"""
import pandas as pd
crit=pd.DataFrame()
crit['Comp']=['H2']
crit['Tc']=[32.938]
crit['Pc']=[1293.00]
crit['w']=[-0.2160]
crit['PM']=[1.00784*2]
"""
class TankCostCalculator:
    price_per_lb_gas = {
        "Type I": 4, #5.5
        "Type II": 9.0,
        "Type III & IV": 15.5
    }

    def __init__(self, tank_type, gas_mass_kg,MMM):
        self.tank_type = tank_type
        self.gas_mass_kg = gas_mass_kg
        self.gas_mass_lb = gas_mass_kg #* 2.20462  # in lb
        self.price_per_lb = self.price_per_lb_gas[tank_type]
        self.extras_df = self.calculate_cost()  # Call calculate_cost during initialization

    def calculate_cost(self):
        CEPCI_2014 = 576.1
        CEPCI_2024 = 798.8
        self.price= (self.gas_mass_lb * self.price_per_lb)* (CEPCI_2024 / CEPCI_2014)
        self.total_material_cost= self.price * 1.2  
        self.construction= self.price * 0.08
        installation_cost = self.price * 2.5
        instrumentation_cost = self.price * 0.08
        Final_cost = self.price + self.total_material_cost + self.construction + installation_cost + instrumentation_cost
        cost_mxn= Final_cost * 19.61  
        self.final=Final_cost
        # Create a data frame with the extra costs
        extras = {
            "Total Price (FOB)": [self.price],
            "Installation Cost (USD)": [installation_cost],
            "Total Material Cost (USD)": [self.total_material_cost],
            "Construction Cost (USD)": [self.construction],
            "Instrumentation Cost (USD)": [instrumentation_cost],
            "Final Cost (USD)": [Final_cost],
            "Final Cost (MXN)": [cost_mxn]
        }
        self.extras_df = pd.DataFrame(extras)
        # Format the numbers in the extras_df as currency
        self.extras_df = self.extras_df.applymap(lambda x: f"${x:,.2f}" if isinstance(x, (int, float)) else x)       
        return  self.extras_df


T1="Type I"
T2="Type II"
T3="Type III & IV"



In [ ]:
Pes=list(np.linspace(70,800,10000))

In [ ]:
m*86400

In [ ]:
Tref=283.15

In [ ]:
def pressure_increment_per_stage(P1, P2, n):
    """
    Calcula el factor de incremento de presión (r) en cada etapa de compresión, y
    devuelve las presiones en cada etapa.
    
    Parámetros:
      P1 : float
          Presión inicial.
      P2 : float
          Presión final.
      n : int
          Número de etapas de compresión.
          
    Retorna:
      r : float
          Factor multiplicativo de presión por etapa.
      pressures : list
          Lista con la presión en cada etapa, desde la inicial hasta la final.
    """
    # Calcular el factor de incremento por etapa
    r = (P2 / P1) ** (1 / n)
    
    # Calcular la presión en cada etapa
    pressures = [P1 * r**i for i in range(n + 1)]
    
    return r, pressures


In [472]:
import pandas as pd
import numpy as np
from scipy.optimize import root

# Inicialización de listas y acumulador de resultados
stages = []
temperature = []
compis = []
Qk = []
Wk = []
waterr = []
price_list = []
resultados_list = []  # Aquí se guardarán los diccionarios con los resultados de cada iteración

for i in range(len(Pes)):
    # Configuración inicial por cada presión en Pes
    P = Pes[i]
    Po = 31
    P_diff = P - Po
    P1_inicial = Po
    P2_target = P

    # Cálculo del número de etapas
    N_etapas= np.ceil(np.log((P2_target) / (Po)) / np.log(1.8))
    stages.append(N_etapas)

    r, pressures = pressure_increment_per_stage(Po, P2_target, int(N_etapas))
    pressures.pop(0)
    # Se calcula el valor “sin límite” para el incremento de presión en cada etapa
    

    # Se inicializa la lista de presiones. El primer valor es la presión base Po.
    P_acum = [Po]
    Temps = [25 + 273.15]

    # Inicialización de contenedores para los datos de cada etapa
    Qs, Ts, m_H2O, comp, power, Ps_list = [], [], [], [], [], []
    Tcomp = []  # Para almacenar "T comp" en cada etapa
    eta = 0.75
    Tref_val = Tref  # Se asume que Tref ya está definida

    # Iterar por cada etapa
    for j in range(int(N_etapas)):
        # Se define, para cada etapa, el incremento permitido:
        if j == 0:
            # En la primera etapa no puede superar 2·Po
            allowed_increment = 2 * Po
        else:
            # En etapas posteriores se toma el incremento usado en la etapa anterior (almacenado en P_acum[-1])
            # y se limita a su doble.
            allowed_increment = 2 * P_acum[-1]

        # Se aplica la restricción: se toma el mínimo entre el valor calculado originalmente y el permitido.
        p_etapa_actual = pressures[j]/2

        # Se calculan las presiones de entrada y salida de la etapa
        P1 = sum(P_acum)
        P_acum.append(p_etapa_actual)
        P2 = sum(P_acum)

        

        # Resolver para el trabajo reversible utilizando root
        x0 = [1.0]
        sol = root(trabajo_reversible_IT, x0=[500], args=(m, P1, P2, eta, 298.15), method='hybr',tol=1e-2)
        if sol.success:
            print(sol.message)
        else:
            print('Error',sol.message)

        # Cálculo del trabajo y otros parámetros
        tkñ = sol.x[0]  # Valor obtenido para T2 (en K)
        print('kkkkkkk', tkñ)
        resultado = trabajo_reversible(m, P1, P2, eta, tkñ - 273.15, 298.15)
        #print(f"W: {resultado:.20f} kW")

        # Resolver la ecuación de balance
        solucion = root(balance, x0, args=(tkñ, Tref_val, m), method='hybr')
        if solucion.success:
            x_encontrado = solucion.x[0]
            

            # Se actualizan los acumuladores de datos de la etapa
            Temps.append(25 + 273.15)
            Qs.append(float(-e))      # Se asume que "e" está definido
            Ts.append(25 + 273.15)
            m_H2O.append(float(x_encontrado))
            comp.append('No. ' + str(j + 1))
            power.append(resultado * 1e3)
            Ps_list.append(P2)
            Tcomp.append(tkñ - 273.15)
        else:
            print("No se encontró solución:", solucion.message)

    # Resto del código (agrupación de resultados, cálculo de costos, etc.)
    data = {
        "Compressor": comp,
        "T (K)": Ts,
        "P (bar)": Ps_list,  # Se asume que el valor está en bar
        "Cooling H2O (kg/s)": m_H2O,
        "Q (kW)": np.array(Qs),
        "W (kW)": np.array(power),
        "T comp": Tcomp
    }
    df_compresores = pd.DataFrame(data)
    calculator = PriceCalculator(df_compresores, N_etapas, CS)
    print(df_compresores)
    total_row = [
        "Total",
        Temps[-1],
        Ps_list[-1],
        sum(df_compresores['Cooling H2O (kg/s)']),
        sum(df_compresores['Q (kW)']),
        sum(df_compresores['W (kW)']),
        sum(df_compresores['T comp']),
        sum(df_compresores['Purchased Equipment Cost (USD)'])
    ]
    df_compresores.loc[df_compresores.shape[0]] = total_row
    compis.append(df_compresores)
    
    # Se instancia el PriceCalculator; se asume que devuelve la información requerida.
    
    calculator_info = {
        "volumen_total": calculator.volumen_total if hasattr(calculator, 'volumen_total') else None,
        "radio":         calculator.radio         if hasattr(calculator, 'radio')         else None,
        "longitud":      calculator.longitud      if hasattr(calculator, 'longitud')      else None,
        "espesor":       calculator.espesor       if hasattr(calculator, 'espesor')       else None,
        "masa":          calculator.masa          if hasattr(calculator, 'masa')          else None,
        "V_objetivo":    calculator.V_objetivo    if hasattr(calculator, 'V_objetivo')    else None,
        "h2":            m  # Se utiliza el valor de m definido externamente
    }
    
    # Actualizar las listas agregadas con información del renglón "Total" de df_compresores
    tot_idx = df_compresores.shape[0] - 1
    temperature.append(df_compresores.loc[tot_idx]['T (K)'])
    Qk.append(df_compresores.loc[tot_idx]['Q (kW)'])
    Wk.append(df_compresores.loc[tot_idx]['W (kW)'])
    waterr.append(df_compresores.loc[tot_idx]['Cooling H2O (kg/s)'])
    
    # Cálculo de dimensiones del tanque y su costo
    resultadot = calcular_dimension_tanque(df_compresores.loc[tot_idx]['P (bar)'], (((m*86400)*1e3)*(4/24))*1.5, crit, 'hemi', 2700)
    calculator1 = TankCostCalculator("Type I", resultadot['masa']*1e3, resultadot['masa']*1e3 )
    demand_example = (df_compresores['W (kW)'].loc[df_compresores.shape[0]-1]) * 1.25        # Por ejemplo, 50 kW base ajustados al 125% → 62.5 kW
    irradiance_example = [737, 763, 748, 702]  # Valores en W/m²
    time_hours_example = 4            # Tiempo de operación en horas
    print('Demand:', demand_example, 'kW')
    resx=compute_pv_system_analysis(demand_example, irradiance_example, PV_eff=0.1, time_hours=time_hours_example)
    
    
    
    total_price =  calculator1.final + calculator.final + resx['Cost']
    price_list.append(total_price)
    # Total de energía consummida por todo el sistemas por 4 horas
    
     
    energy=resx['Ps_inst_kW'] #kw/day
    #convert kw/year to MW/day
    energy_total_mw=energy/1000 #MW/dia

    FESEN = 0.444 #tCO2e/MWh #RED ELÉCTRICA
    FESENSOL = 0.180 #tCO2e/MWh #Energía solar
    FESENSTEEL = 2.1 #tCO2e/tonn

    #calcculate the total emissions in tCO2e/year
    emissions1 = energy_total_mw *FESEN*365 #tCO2e/year
    emissions2 = energy_total_mw *FESENSOL*365 #tCO2e/year
    emissions3 = resultadot['masa'] *FESENSTEEL #tCO2e/year
    
    
    # Crear un diccionario que combine los datos totales y la info de calculator
    resultado_row = {
        "T (K)": df_compresores.loc[tot_idx]['T (K)'],
        "P (bar)": df_compresores.loc[tot_idx]['P (bar)'],
        "Cooling H2O (kg/s)": df_compresores.loc[tot_idx]['Cooling H2O (kg/s)'],
        "Q (kW)": df_compresores.loc[tot_idx]['Q (kW)'],
        "W (kW)": df_compresores.loc[tot_idx]['W (kW)'],
        "volumen_total": calculator_info["volumen_total"],
        "radio":         calculator_info["radio"],
        "longitud":      calculator_info["longitud"],
        "espesor":       calculator_info["espesor"],
        "masa":          calculator_info["masa"],
        "V_objetivo":    calculator_info["V_objetivo"],
        "h2":            calculator_info["h2"],
        "SOLAR":         resx['Cost'],
        "price":         total_price,
        "CO2e":          1e3*(emissions3+emissions2),
        "CO2e (solar)":  emissions2+emissions3,
        "CO2e (CFE)":    emissions1+emissions3,
        "It. No.":       i
    }
    resultados_list.append(resultado_row)

# Crear el DataFrame final de resultados a partir de la lista
df_resultados = pd.DataFrame(resultados_list)
df_resultados

Volumen calculado del tanque: 87.2 m³
Radio del tanque: 2.03 m
Longitud del tanque: 8.11 m
Espesor del tanque: 151.47 mm
Masa del tanque: 207.79 ton
Demand: 224.44205114039346 kW
Producción: 737.5 kW, Demanda: 224.44205114039346 kW
Producción: 737.5 kW, Demanda: 224.44205114039346 kW
Producción: 737.5 kW, Demanda: 224.44205114039346 kW
Producción: 737.5000109896064 kW, Demanda: 224.44205114039346 kW
Producción: 224.44205113927438 kW, Demanda: 224.44205114039346 kW
Producción: 224.44205114039346 kW, Demanda: 224.44205114039346 kW
kW 77.7271414293757
kW 77.7271414293757
kW 77.7271414293757
kW 77.72714208249168
kW 65.81861487163594
kW 65.45512110486699
kW 65.44404872827478
The solution converged.
kkkkkkk 368.80669030733236
kW 68.41070803748693
kW 68.41070803748693
kW 68.41070803748693
kW 68.4107086226821
kW 56.7925997213936
kW 56.40333475937282
kW 56.39035318721525
The solution converged.
kkkkkkk 359.10272892493714
kW 63.30989268290662
kW 63.30989268290662
kW 63.30989268290662
kW 63.30989

KeyboardInterrupt: 

In [ ]:
m*86400

In [ ]:
%pip install SciencePlots

import matplotlib.pyplot as plt
import scienceplots  # Asegúrate de tener instalado: SciencePlots
import math

# Desactivar el uso de LaTeX para evitar el error.
#plt.rcParams['text.usetex'] = False

# Utiliza una combinación de estilos populares para obtener un formato científico
#plt.style.use(['science'])#, 'ieee', 'std-colors'])

# Lista de variables a graficar versus "P (bar)"
variables_to_plot = [
    "T (K)",
    "Cooling H2O (kg/s)",
    "Q (kW)",
    "W (kW)",
    "volumen_total",
    "radio",
    "longitud",
    "espesor",
    "masa",
    "V_objetivo",
    "h2",
    "price"
]

# Se asume que df_resultados ya está definido y contiene la columna "P (bar)" y las demás variables
df_resultados_sorted = df_resultados.sort_values(by="P (bar)")
import pandas as pd

# Se asume que df_resultados_sorted ya está definido y contiene la columna 'price'

# Calcular el primer (Q1) y tercer cuartil (Q3)
q1 = df_resultados_sorted['price'].quantile(0.25)
q3 = df_resultados_sorted['price'].quantile(0.75)

# Calcular el rango intercuartílico (IQR)
IQR = q3 - q1

# Definir los límites inferior y superior para detectar outliers
lower_bound = q1 - 1.5 * IQR
upper_bound = q3 + 1.5 * IQR

# Filtrar el DataFrame eliminando los valores atípicos en la columna 'price'
df_resultados_filtrado = df_resultados_sorted[
    (df_resultados_sorted['price'] >= lower_bound) &
    (df_resultados_sorted['price'] <= upper_bound)
]
df_resultados_sorted=df_resultados_sorted
# Opcional: Mostrar el DataFrame filtrado


num_vars = len(variables_to_plot)
cols = 3
rows = math.ceil(num_vars / cols)

fig, axes = plt.subplots(rows, cols, figsize=(6 * cols, 4 * rows), constrained_layout=True)
axes = axes.flatten()

# Graficar cada variable versus "P (bar)"
for i, var in enumerate(variables_to_plot):
    ax = axes[i]
    ax.plot(
        df_resultados_sorted["P (bar)"],
        df_resultados_sorted[var],
        marker="o", linestyle="-", color="black"
    )
    ax.set_xlabel("P (bar)")
    ax.set_ylabel(var)
    ax.set_title(f"{var} vs P (bar)")
    ax.grid(True)
    ax.tick_params(axis="both", which="both", direction="in", top=True, right=True)

# Eliminar axes vacíos en caso de que excedan el total de variables
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.show()


In [ ]:
mini=df_resultados_sorted[df_resultados_sorted['price']==df_resultados_sorted['price'].min()]
mini

In [ ]:
minis=df_resultados_sorted[df_resultados_sorted['CO2e (solar)']==df_resultados_sorted['CO2e (solar)'].min()]
minis

In [ ]:
minic=df_resultados_sorted[df_resultados_sorted['CO2e (CFE)']==df_resultados_sorted['CO2e (CFE)'].min()]
minic

In [ ]:
kj=compis[mini.index[0]]
kj

In [ ]:
kj['Purchased Equipment Cost (USD)']

In [ ]:
data=pd.read_csv('data copy.csv', sep=',', encoding='utf-8')
data

In [ ]:
k=compis[df_resultados['It. No.'][mini.index[0]]]
k['Cooling H2O (ton/d)']=(k['Cooling H2O (kg/s)']*86400)/1e3*(4/24) #ton
k

In [ ]:
import matplotlib.pyplot as plt
#import scienceplots  # Asegúrate de tener instalado: pip install SciencePlots

# Opcional: activar un estilo popular (en este ejemplo usamos 'no-latex')
plt.style.use(['no-latex'])

# Se asume que df_resultados ya está definido y contiene las columnas "P (bar)" y "price"
#df_resultados_sorted = df_resultados.sort_values(by="P (bar)")

# Crear la figura y el eje
fig, ax = plt.subplots(figsize=(8, 6))

# Graficar 'price' en función de 'P (bar)'
ax.plot(df_resultados_sorted["P (bar)"], df_resultados_sorted["CO2e"],
        linestyle="-", color="black",lw=1.5)

# Personalizar etiquetas y título
ax.set_xlabel("P (bar)", fontsize=14)
ax.set_ylabel("Area (m2)", fontsize=14)
ax.set_title("Photovoltaic system area as function of storage pressure", fontsize=19)

# Activar la grilla y ajustar los ticks para un formato profesional

ax.tick_params(axis="both", which="major", labelsize=14, direction="in", top=True, right=True)
ax.tick_params(axis="both", which="minor", labelsize=12, direction="in", top=True, right=True)

# Forzar notación científica (esto hará que se muestre la notación ×10^...)
ax.ticklabel_format(axis="both", style="sci")

# Aumentar el tamaño de la fuente de la notación científica (offset text)
ax.xaxis.get_offset_text().set_fontsize(14)
ax.yaxis.get_offset_text().set_fontsize(14)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
#import scienceplots  # Asegúrate de tener instalado: pip install SciencePlots

# Opcional: activar un estilo popular (en este ejemplo usamos 'no-latex')
plt.style.use(['no-latex'])

# Se asume que df_resultados ya está definido y contiene las columnas "P (bar)" y "price"
#df_resultados_sorted = df_resultados.sort_values(by="P (bar)")

# Crear la figura y el eje
fig, ax = plt.subplots(figsize=(8, 6))

# Graficar 'price' en función de 'P (bar)'
ax.plot(df_resultados_sorted["P (bar)"], df_resultados_sorted["price"],
        linestyle="-", color="black",lw=1.5)

# Personalizar etiquetas y título
ax.set_xlabel("P (bar)", fontsize=14)
ax.set_ylabel("Price (USD)", fontsize=14)
ax.set_title("Storage tanks installation price as function of storage pressure", fontsize=19)

# Activar la grilla y ajustar los ticks para un formato profesional

ax.tick_params(axis="both", which="major", labelsize=14, direction="in", top=True, right=True)
ax.tick_params(axis="both", which="minor", labelsize=12, direction="in", top=True, right=True)

# Forzar notación científica (esto hará que se muestre la notación ×10^...)
ax.ticklabel_format(axis="both", style="sci")

# Aumentar el tamaño de la fuente de la notación científica (offset text)
ax.xaxis.get_offset_text().set_fontsize(14)
ax.yaxis.get_offset_text().set_fontsize(14)

plt.show()


In [ ]:
m1=m*3600*4*365*1e3
m1

In [ ]:
import matplotlib.pyplot as plt

# Opcional: activar un estilo popular
plt.style.use(['no-latex'])

# Se asume que df_resultados ya está definido y contiene las columnas "P (bar)", "price" y "CO2e"
df_resultados_sorted = df_resultados.sort_values(by="P (bar)")

# Crear la figura y el eje principal
fig, ax1 = plt.subplots(figsize=(8, 6))

# Graficar 'price' en función de 'P (bar)' en el eje izquierdo
ax1.plot(df_resultados_sorted["P (bar)"], df_resultados_sorted["price"]/m1,
         linestyle="-", color="black", lw=1.5, label="Price (USD)")
ax1.set_xlabel("P (bar)", fontsize=14)
ax1.set_ylabel("USD/kgH2", fontsize=14, color="black")
ax1.tick_params(axis="y", labelcolor="black")

# Crear un segundo eje y para 'CO2e'
ax2 = ax1.twinx()
ax2.plot(df_resultados_sorted["P (bar)"], df_resultados_sorted["CO2e"]/m1,
         linestyle="-", color="red", lw=1.5, label="CO2e (ton)")
ax2.set_ylabel("kgCO2e/kgH2", fontsize=14, color="red")
ax2.tick_params(axis="y", labelcolor="red")

# Personalizar título y formato
ax1.set_title("Overall installation price & CO2e as function of storage pressure", fontsize=15)
ax1.tick_params(axis="both", which="major", labelsize=14, direction="in", top=True, right=True)
ax1.tick_params(axis="both", which="minor", labelsize=12, direction="in", top=True, right=True)

ax2.tick_params(axis="both", which="major", labelsize=14, direction="in", top=True, right=True)
ax2.tick_params(axis="both", which="minor", labelsize=12, direction="in", top=True, right=True)

# Forzar notación científica
ax1.ticklabel_format(axis="y", style="sci")
ax2.ticklabel_format(axis="y", style="sci")

# Aumentar el tamaño de la fuente de la notación científica
ax1.yaxis.get_offset_text().set_fontsize(14)
ax2.yaxis.get_offset_text().set_fontsize(14)

plt.show()

In [ ]:
import pandas as pd

data = {
    "Fuel": ["Hydrogen", "Methanol", "Methane", "Ethanol", "Propane", "Butane", "Octane"],
    "Formula": ["H2(g)", "CH3OH(l)", "CH4(g)", "CH3CH2OH(l)", "C3H8(g)", "C4H10(l)", "C8H18(l)"],
    "Heat of Combustion (kJ/g)": [142, 23, 56, 30, 50, 50, 48]
}

df = pd.DataFrame(data)
df["Heat of Combustion (kJ/g)"]*1e3


In [ ]:
import pandas as pd

# Definición de los datos según la información de la imagen
data = {
    "Fuel": ["Hydrogen", "Methanol", "Methane", "Ethanol", "Propane", "Butane", "Octane"],
    "Formula": ["H₂(g)", "CH₃OH(l)", "CH₄(g)", "CH₃CH₂OH(l)", "C₃H₈(g)", "C₄H₁₀(l)", "C₈H₁₈(l)"],
    "Heat of Combustion (kJ/mol)": [-286, -726, -891, -1367, -2219, -2878, -5471],
    "Heat of Combustion (kJ/g)": [-142, -23, -56, -30, -50, -50, -48]
}

# Creación del DataFrame
df = pd.DataFrame(data)

# Visualización del DataFrame
print(df)


In [ ]:
import pandas as pd

# Definimos los datos en un diccionario
data = {
    "Fuel Type": ["H2","Coal", "Coke", "Crude Oil", "Gasoline", "Kerosene",
                  "Diesel Oil", "Fuel Oil", "Natural Gas"],
    "LCV (KJ/kg or KJ/m³)": [286*1e3,20908, 28435, 41816, 43070, 43070, 42652, 41816, 38931],
    "CO2 EF (tCO2/t or 10³ m³)": [3.3,1.981, 2.860, 3.120, 2.925, 3.025, 3.120, 3.170, 2.162]
}

# Creamos el DataFrame a partir del diccionario
df = pd.DataFrame(data)

# Mostramos la tabla
df['kj/kgCO2']=df['LCV (KJ/kg or KJ/m³)']/df['CO2 EF (tCO2/t or 10³ m³)']
df


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Activar el estilo (si ya lo tienes instalado o definido)
plt.style.use(['no-latex'])

# Definimos los datos en un diccionario
data = {
    "Fuel Type": ["H2", "Coal", "Coke", "Crude Oil", "Gasoline", "Kerosene",
                  "Diesel Oil", "Fuel Oil", "Natural Gas"],
    "LCV (KJ/kg or KJ/m³)": [286 * 1e3, 20908, 28435, 41816, 43070, 43070, 42652, 41816, 38931],
    "CO2 EF (tCO2/t or 10³ m³)": [0.7, 1.981, 2.860, 3.120, 2.925, 3.025, 3.120, 3.170, 2.162]
}

# Crear el DataFrame a partir del diccionario
df = pd.DataFrame(data)

# Calcular la razón KJ/CO2 y añadirla como nueva columna
df['kj/kgCO2'] = df['LCV (KJ/kg or KJ/m³)'] / df['CO2 EF (tCO2/t or 10³ m³)']
df.sort_values(by='kj/kgCO2',ascending=False,inplace=True)

# Crear la figura y el eje principal
fig, ax = plt.subplots(figsize=(10, 6))

# Graficar las barras con un color de relleno y borde para destacar cada barra
bars = ax.bar(df["Fuel Type"], df['kj/kgCO2'], color="darkgrey", edgecolor="black", linewidth=1.2)

# Añadir etiquetas y título con el tamaño de fuente especificado
ax.set_xlabel("Fuel Type", fontsize=14)
ax.set_ylabel("kJ/CO2e", fontsize=14)
ax.set_title("Fuel Type vs KJ/CO2e", fontsize=15)

# Personalizar los parámetros de los ticks de los ejes para mejorar la legibilidad
ax.tick_params(axis="both", which="major", labelsize=14, direction="in", top=True, right=True)
ax.tick_params(axis="both", which="minor", labelsize=12, direction="in", top=True, right=True)

# Forzar notación científica en el eje y (opcional)
ax.ticklabel_format(axis="y", style="sci")
ax.yaxis.get_offset_text().set_fontsize(14)

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Definimos los datos en un diccionario
data = {
    "Fuel Type": ["H2", "Coal", "Coke", "Crude Oil", "Gasoline", "Kerosene",
                  "Diesel Oil", "Fuel Oil", "Natural Gas"],
    "LCV (KJ/kg or KJ/m³)": [286*1e3, 20908, 28435, 41816, 43070, 43070, 42652, 41816, 38931],
    "CO2 EF (tCO2/t or 10³ m³)": [0.7, 1.981, 2.860, 3.120, 2.925, 3.025, 3.120, 3.170, 2.162]
}

# Creamos el DataFrame a partir del diccionario
df = pd.DataFrame(data)

# Calculamos la razón KJ/CO2
df['kj/kgCO2'] = df['LCV (KJ/kg or KJ/m³)'] / df['CO2 EF (tCO2/t or 10³ m³)']

# Configuramos el gráfico de barras
plt.figure(figsize=(10, 6))
plt.bar(df["Fuel Type"], df['kj/kgCO2'], color='skyblue')
plt.xlabel("Fuel Type")
plt.ylabel("KJ/CO2")
plt.title("Fuel Type vs KJ/CO2")
plt.xticks(rotation=45)  # Rotamos las etiquetas del eje X para mayor claridad
plt.tight_layout()
plt.show()


In [ ]:
r=df_resultados_sorted["price"]/m1
r.min()

In [ ]:
800/2204.62

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Emission factors in tCO2e/MWh
FESEN = 0.444  # Electrical grid
FESENSOL = 0.180  # Solar energy

# Assumed total energy generated (MW)
energy_total_mw = 100  # Adjust as needed

# Calculate annual emissions
emissions1_yearly = energy_total_mw * FESEN * 365  # tCO2e/year
emissions2_yearly = energy_total_mw * FESENSOL * 365  # tCO2e/year

# Create the range of years
years = np.arange(1, 11)  # From 1 to 10 years

# Calculate accumulated emissions over time
emissions1_total = emissions1_yearly * years
emissions2_total = emissions2_yearly * years

# Create the figure and axis
fig, ax = plt.subplots(figsize=(8, 6))

# Plot both technologies under the same y-axis
ax.plot(years, emissions1_total, linestyle="-", color="black", lw=1.5, label="Electrical Grid")
ax.plot(years, emissions2_total, linestyle="-", color="green", lw=1.5, label="Solar Energy")

# Customize labels and title
ax.set_xlabel("Time (years)", fontsize=14)
ax.set_ylabel("Accumulated Emissions (tCO2e)", fontsize=14)
ax.set_title("Comparison of Accumulated Emissions Over 10 Years", fontsize=19)

# Enable the legend
ax.legend()

# Enable grid for better visualization
ax.grid(True)

plt.show()

In [ ]:
(k['W (kW)'].loc[k.shape[0]-1]) * 1.25

In [473]:
import numpy as np
from scipy.integrate import quad
from scipy.optimize import fsolve

def find_required_area(demand, irradiance_values, PV_eff=0.1):
    """
    Encuentra el área A (en m²) requerida para que la potencia efectiva de la planta PV
    sea igual a la demanda.
    
    Parámetros:
      demand             : Demanda en kW (por ejemplo, 50*1.25)
      irradiance_values  : Lista o array de valores de irradiancia en W/m².
      PV_eff             : Eficiencia del sistema fotovoltaico (por defecto 0.1 = 10%)
    
    Retorna:
      A_required         : Área requerida en m², tal que:
                           (Promedio(irradiancia)*A_required/1000)*PV_eff = demand.
    """
    # Calcular la irradiancia promedio (W/m²)
    Is_mean = np.mean(irradiance_values)
    
    # Función objetivo: (Is_mean * A / 1000) * PV_eff debe ser igual a demand
    def objective(A):
        production = (Is_mean * A[0] / 1000) * PV_eff  # en kW
        print(f"Producción: {production} kW, Demanda: {demand} kW")
        return production - demand

    # Valor inicial (por ejemplo, 1000 m²)
    A_initial = 10000
    A_solution = opt.root(objective, A_initial, 
                           method='hybr',tol=1e-2, options={'maxfev': 1000000})

    if A_solution.success:
        return A_solution.x[0]
    else:
        raise RuntimeError("No se encontró la solución: " + A_solution.message)
    

def compute_pv_system_analysis(demand, irradiance_values, PV_eff=0.1, time_hours=4):
    """
    Realiza el análisis completo de un sistema PV: optimiza el área requerida y calcula
    diversas energías (producción, integración, acoplamiento entre generación y demanda).
    
    Parámetros:
      demand             : Demanda en kW (ya ajustada, por ejemplo, demand = 50*1.25)
      irradiance_values  : Lista o array de valores de irradiancia en W/m².
      PV_eff             : Eficiencia del sistema PV (por defecto 0.1, es decir, 10%).
      time_hours         : Tiempo de operación en horas para el que se calcula la energía.
      
    Retorna:
      results (dict)     : Diccionario con los siguientes valores:
           - "Area_required_m2"       : Área requerida (m²).
           - "demand_kW"              : Demanda en kW.
           - "irradiance_mean_W_m2"   : Irradiancia media en W/m².
           - "Ps_inst_kW"             : Potencia solar instantánea teórica en kW.
           - "Pv_kW"                  : Potencia efectiva PV en kW.
           - "E_pv_constant_kWh"      : Energía producida suponiendo potencia constante (kWh).
           - "E_demand_integrated_kWh": Energía demandada integrada (kWh).
           - "err_demand"             : Error estimado en la integración de la demanda.
           - "E_pv_integrated_kWh"    : Energía producida (integración con potencia constante) (kWh).
           - "err_PV"                 : Error estimado en la integración de la PV (constante).
           - "E_pv_linear_kWh"        : Energía producida considerando un perfil lineal (kWh).
           - "err_PV_linear"          : Error estimado en la integración del perfil lineal.
           - "Esupplied_kWh"          : Energía suministrada efectivamente (kWh).
           - "Eunsupplied_kWh"        : Energía no suministrada (kWh).
           - "Eunused_kWh"            : Energía solar no utilizada (kWh).
    """
    # Optimización: encontrar el área requerida para que la producción PV iguale a la demanda
  

    A_required = find_required_area(demand, irradiance_values, PV_eff)
    
    # Cálculo de parámetros de irradiancia y potencia
    Is_mean = np.mean(irradiance_values)  # en W/m²
    Ps_inst = (Is_mean * A_required) / 1000  # Potencia solar instantánea en kW
    Pv = Ps_inst * PV_eff  # Potencia efectiva de la planta PV en kW

    # Energía eléctrica producida durante el período (potencia constante)
    E_pv = Pv * time_hours  # en kWh

    # =============================================================================
    # Ejemplos de integración con scipy.quad
    # =============================================================================
    # (A) Energía demandada (suponiendo demanda constante)
    def constant_demand(x):
        return demand  # demanda en kW (constante)
    El, err_demand = quad(constant_demand, 0, time_hours)  # Resultado en kWh

    # (B) Energía producida por el sistema PV (suponiendo potencia constante)
    def constant_PV(x):
        return Pv  # potencia PV en kW (constante)
    E_pv_integrated, err_PV = quad(constant_PV, 0, time_hours)  # Resultado en kWh

    # (C) Energía producida con perfil lineal (de 0 a Pv)
    def linear_PV(x):
        return (Pv / time_hours) * x
    E_pv_linear, err_PV_linear = quad(linear_PV, 0, time_hours)  
    # Teóricamente, para perfil lineal: E = 0.5 * Pv * time_hours

    # =============================================================================
    # Cálculo del acoplamiento entre generación y demanda
    # =============================================================================
    if Pv > demand:
        Esup = demand * time_hours   # La planta puede cubrir toda la demanda
    else:
        Esup = Pv * time_hours        # Solo se suministra lo que produce la planta

    if Pv < demand:
        Eunsup = (demand - Pv) * time_hours  # Energía no suministrada
    else:
        Eunsup = 0

    if Pv > demand:
        Eunused = (Pv - demand) * time_hours  # Energía solar no utilizada
    else:
        Eunused = 0


    price = 755
    price = price * (293.9/239.1)
    SPP = price * Ps_inst
    cost = SPP * 1.35
    # Compilamos los resultados en un diccionario
    results = {
        "Area_required_m2": A_required,
        "demand_kW": demand,
        "irradiance_mean_W_m2": Is_mean,
        "Ps_inst_kW": Ps_inst,
        "Pv_kW": Pv,
        "E_pv_constant_kWh": E_pv,
        "E_demand_integrated_kWh": El,
        "err_demand": err_demand,
        "E_pv_integrated_kWh": E_pv_integrated,
        "err_PV": err_PV,
        "E_pv_linear_kWh": E_pv_linear,
        "err_PV_linear": err_PV_linear,
        "Esupplied_kWh": Esup,
        "Eunsupplied_kWh": Eunsup,
        "Eunused_kWh": Eunused,
        "Cost": cost
    }
    
    # Impresión de resultados
    print("=== Análisis de Unidades y Resultados ===")
    print(f"Área requerida (m²) para que la producción PV iguale la demanda: {results['Area_required_m2']:.2f} m²")
    print(f"Demanda (constante): {results['demand_kW']:.2f} kW")
    print(f"Irradiancia media: {results['irradiance_mean_W_m2']:.2f} W/m²")
    print(f"Potencia solar instantánea teórica: {results['Ps_inst_kW']:.2f} kW")
    print(f"Potencia efectiva PV (con eficiencia del {PV_eff*100:.0f}%): {results['Pv_kW']:.2f} kW")
    print(f"Energía producida por la planta PV en {time_hours} horas (constante): {results['E_pv_constant_kWh']:.2f} kWh")
    print(f"Energía demandada (integrada): {results['E_demand_integrated_kWh']:.2f} kWh (error: {results['err_demand']:.2e})")
    print(f"Energía PV (integrada, constante): {results['E_pv_integrated_kWh']:.2f} kWh (error: {results['err_PV']:.2e})")
    print(f"Energía PV (con perfil lineal): {results['E_pv_linear_kWh']:.2f} kWh (error: {results['err_PV_linear']:.2e})")
    print(f"Energía suministrada por el sistema PV: {results['Esupplied_kWh']:.2f} kWh")
    print(f"Energía no suministrada: {results['Eunsupplied_kWh']:.2f} kWh")
    print(f"Energía solar no utilizada: {results['Eunused_kWh']:.2f} kWh")
    print(f"Coste estimado del sistema: ${results['Cost']:.2f} USD")
    
    return results

# ---------------------------------------------------------------------------
# Ejemplo de uso:
# Supongamos que la demanda se obtiene de un DataFrame "k" y se ajusta con 1.25.
# Por ejemplo:
#   demand = (k['W (kW)'].iloc[-1]) * 1.25
# Para este ejemplo, definimos una demanda base arbitraria:

demand_example = 542.67459     # Por ejemplo, 50 kW base ajustados al 125% → 62.5 kW
irradiance_example = [737, 763, 748, 702]  # Valores en W/m²
time_hours_example = 4            # Tiempo de operación en horas
    
resultado=compute_pv_system_analysis(demand_example, irradiance_example, PV_eff=0.1, time_hours=time_hours_example)
resultado


Producción: 737.5 kW, Demanda: 542.67459 kW
Producción: 737.5 kW, Demanda: 542.67459 kW
Producción: 737.5 kW, Demanda: 542.67459 kW
Producción: 737.5000109896064 kW, Demanda: 542.67459 kW
Producción: 542.674589999575 kW, Demanda: 542.67459 kW
Producción: 542.67459 kW, Demanda: 542.67459 kW
=== Análisis de Unidades y Resultados ===
Área requerida (m²) para que la producción PV iguale la demanda: 7358.30 m²
Demanda (constante): 542.67 kW
Irradiancia media: 737.50 W/m²
Potencia solar instantánea teórica: 5426.75 kW
Potencia efectiva PV (con eficiencia del 10%): 542.67 kW
Energía producida por la planta PV en 4 horas (constante): 2170.70 kWh
Energía demandada (integrada): 2170.70 kWh (error: 2.41e-11)
Energía PV (integrada, constante): 2170.70 kWh (error: 2.41e-11)
Energía PV (con perfil lineal): 1085.35 kWh (error: 1.20e-11)
Energía suministrada por el sistema PV: 2170.70 kWh
Energía no suministrada: 0.00 kWh
Energía solar no utilizada: 0.00 kWh
Coste estimado del sistema: $6798924.47 USD

{'Area_required_m2': 7358.2995254237285,
 'demand_kW': 542.67459,
 'irradiance_mean_W_m2': 737.5,
 'Ps_inst_kW': 5426.7459,
 'Pv_kW': 542.67459,
 'E_pv_constant_kWh': 2170.69836,
 'E_demand_integrated_kWh': 2170.69836,
 'err_demand': 2.4099592987880667e-11,
 'E_pv_integrated_kWh': 2170.69836,
 'err_PV': 2.4099592987880667e-11,
 'E_pv_linear_kWh': 1085.34918,
 'err_PV_linear': 1.2049796493940334e-11,
 'Esupplied_kWh': 2170.69836,
 'Eunsupplied_kWh': 0,
 'Eunused_kWh': 0,
 'Cost': 6798924.474885791}

In [ ]:
resultado['Cost']

In [ ]:
import numpy as np
from scipy.integrate import quad
from scipy.optimize import fsolve

def find_required_area(demand, irradiance_values, PV_eff=0.1):
    """
    Encuentra el área A (en m²) requerida para que la potencia efectiva de la planta PV
    sea igual a la demanda.

    Parámetros:
      demand             : Demanda en kW (por ejemplo, 50*1.25)
      irradiance_values  : Lista o array de valores de irradiancia en W/m².
      PV_eff             : Eficiencia del sistema fotovoltaico (por defecto 0.1 = 10%)

    Retorna:
      A_required         : Área requerida en m², tal que
                           (Promedio(irradiancia)*A_required/1000)*PV_eff = demand.
    """
    # Calcular la irradiancia promedio (W/m²)
    Is_mean = np.mean(irradiance_values)
    
    # Definir la función objetivo:
    # Queremos que: effective_PV = (Is_mean*A/1000)*PV_eff - demand = 0
    def objective(A):
        production = (Is_mean * A / 1000) * PV_eff  # en kW
        return production - demand

    # Valor inicial (por ejemplo, 1000 m²)
    A_initial = 1000.0
    A_solution, info, ier, mesg = fsolve(objective, A_initial, full_output=True)
    
    if ier != 1:
        raise RuntimeError("No se encontró la solución: " + mesg)
    return A_solution[0]

# =============================================================================
# Parámetros de entrada
# =============================================================================

demand = (k['W (kW)'].loc[k.shape[0]-1]) * 1.25  # en kW → 62.5 kW

# Valores de irradiancia medidos (en W/m²)
irradiance_values = [737, 763, 748, 702]

# Eficiencia del sistema fotovoltaico (10% en este ejemplo)
PV_eff = 0.1

# Tiempo de operación para integración (en horas)
time_hours = 4

# =============================================================================
# Optimización: Encontrar el área requerida para que PV = demand
# =============================================================================

A_required = find_required_area(demand, irradiance_values, PV_eff)


# =============================================================================
# Cálculo de la potencia teórica y la energía producida (usando la solución óptima)
# =============================================================================

# Potencia solar teórica (disponible) en kW:
Is_mean = np.mean(irradiance_values)  # en W/m²
Ps_inst = (Is_mean * A_required) / 1000  # en kW

# Potencia efectiva PV (aplicando la eficiencia)
Pv = Ps_inst * PV_eff  # en kW (idealmente igual a la demanda)

# Si se desea calcular la energía producida durante 'time_hours', 
# suponiendo potencia constante, la energía es:
E_pv = Pv * time_hours  # en kWh

# =============================================================================
# Cálculo del acoplamiento entre generación y demanda
# =============================================================================
#
# La energía suministrada (Esup) es el mínimo entre la energía producida por el sistema 
# y la demanda total, asumiendo un sistema de autoconsumo sin almacenamiento.
if Pv > demand:
    Esup = demand * time_hours   # La planta puede cubrir toda la demanda
else:
    Esup = Pv * time_hours        # Solo se suministra lo que produce la planta

# Energía no suministrada (si la producción es menor que la demanda)
if Pv < demand:
    Eunsup = (demand - Pv) * time_hours  # en kWh
else:
    Eunsup = 0

# Energía solar no utilizada (si la producción excede la demanda)
if Pv > demand:
    Eunused = (Pv - demand) * time_hours  # en kWh
else:
    Eunused = 0

# =============================================================================
# Impresión de resultados
# =============================================================================

print("=== Análisis de Unidades y Resultados ===")
print(f"Área requerida (m²) para que la producción PV iguale la demanda: {A_required:.2f}")
print(f"Demanda (constante): {demand:.2f} kW")
print(f"Potencia efectiva PV (kW): {Pv:.2f}")
print(f"Energía producida por la planta PV en {time_hours} horas (kWh): {E_pv:.2f}")
print(f"Irradiancia media: {Is_mean:.2f} W/m²")
print(f"Potencia solar instantánea teórica: {Ps_inst:.2f} kW")
print(f"Potencia efectiva PV (con eficiencia del {PV_eff*100:.0f}%): {Pv:.2f} kW")
print(f"Energía suministrada por el sistema PV: {Esup:.2f} kWh")
print(f"Energía no suministrada: {Eunsup:.2f} kWh")
print(f"Energía solar no utilizada: {Eunused:.2f} kWh")



In [ ]:
k['W (kW)'].loc[k.shape[0]-1]

In [ ]:
import numpy as np
from scipy.integrate import quad

# =============================================================================
# Parámetros de entrada
# =============================================================================

# SUPOSICIÓN: La variable "k" es un DataFrame que contiene la demanda en kW, 
# por ejemplo, en una columna llamada "W (kW)". Si no está definido, aquí 
# usamos un ejemplo: suponemos que la demanda final es de 50 kW, y se ajusta en 1.25.
# Para trabajar en kW, eliminamos la conversión a Watts (multiplicar por 1e3).
demand = (k['W (kW)'].loc[k.shape[0]-1]*1.25)*1e3 * 1.25     # Demanda en kW (50 kW de base, ajustados al 125%)

time_hours = 4         # Tiempo de operación en horas

# =============================================================================
# Cálculo de la irradiancia y potencia disponible
# =============================================================================

# Se considera una serie de valores medidos de irradiancia [W/m²]
irradiance_values = [737, 763, 748, 702]
Is_mean = np.mean(irradiance_values)  # Promedio en W/m²

# Área del generador (o colector) en m²
A = 3000  # m²

# Potencia solar instantánea teórica que incide sobre el área (en W) y convertida a kW:
#   P_inst (W) = irradiancia (W/m²) * área (m²)
#   P_inst (kW) = (irradiancia * área) / 1000
Ps_inst = (Is_mean * A) / 1000  # en kW

# También podemos calcular la energía total incidente sobre la superficie a lo largo del período:
#   E_incidente (kWh) = (irradiancia [W/m²] * tiempo [h] * área [m²]) / 1000
E_incident = (Is_mean * time_hours * A) / 1000  # en kWh

# =============================================================================
# Cálculo de la energía producida por el sistema PV
# =============================================================================

PV_eff = 0.1           # Eficiencia del sistema PV (10%)
Pv = Ps_inst * PV_eff  # Potencia efectiva producida por la planta PV en kW

# La energía eléctrica producida durante el período (suponiendo que la potencia es constante)
E_pv = Pv * time_hours  # en kWh

# =============================================================================
# Ejemplos de integración con scipy.quad
# =============================================================================
#
# En nuestro caso, si el perfil de demanda o de producción fuera constante, la integración
# es directa. Se muestra cómo usar quad para funciones constantes.
#
# (A) Energía demandada:
def constant_demand(x):
    return demand  # demanda en kW (constante)

El, err_demand = quad(constant_demand, 0, time_hours)  # Resultado en kWh

# (B) Energía producida por el sistema PV (suponiendo potencia constante):
def constant_PV(x):
    return Pv  # potencia PV en kW (constante)

E_pv_integrated, err_PV = quad(constant_PV, 0, time_hours)  # Resultado en kWh

# (C) Ejemplo: Si la potencia de la planta aumenta linealmente de 0 a Pv a lo largo del tiempo.
def linear_PV(x):
    # Potencia lineal: P(x) = (Pv / time_hours) * x.
    return (Pv / time_hours) * x

E_pv_linear, err_PV_linear = quad(linear_PV, 0, time_hours)  
# Con un perfil lineal, la energía total es: 0.5 * Pv * time_hours

# =============================================================================
# Cálculo del acoplamiento entre generación y demanda
# =============================================================================
#
# La energía suministrada (Esup) es el mínimo entre la energía producida por el sistema 
# y la demanda total, asumiendo un sistema de autoconsumo sin almacenamiento.
if Pv > demand:
    Esup = demand * time_hours   # La planta puede cubrir toda la demanda
else:
    Esup = Pv * time_hours        # Solo se suministra lo que produce la planta

# Energía no suministrada (si la producción es menor que la demanda)
if Pv < demand:
    Eunsup = (demand - Pv) * time_hours  # en kWh
else:
    Eunsup = 0

# Energía solar no utilizada (si la producción excede la demanda)
if Pv > demand:
    Eunused = (Pv - demand) * time_hours  # en kWh
else:
    Eunused = 0

# =============================================================================
# Impresión de resultados
# =============================================================================

print("=== Análisis de Unidades y Resultados ===")
print(f"Demanda (constante): {demand:.2f} kW")
print(f"Total de energía demandada (integrada): {El:.2f} kWh (error: {err_demand:.2e})")
print(f"Irradiancia media: {Is_mean:.2f} W/m²")
print(f"Potencia solar instantánea teórica: {Ps_inst:.2f} kW")
print(f"Energía solar incidente sobre el área: {E_incident:.2f} kWh")
print(f"Potencia efectiva PV (con eficiencia del {PV_eff*100:.0f}%): {Pv:.2f} kW")
print(f"Energía producida por el sistema PV (constante): {E_pv:.2f} kWh")
print(f"Energía PV (integrada, constante): {E_pv_integrated:.2f} kWh (error: {err_PV:.2e})")
print(f"Energía PV (con perfil lineal): {E_pv_linear:.2f} kWh (error: {err_PV_linear:.2e})")
print(f"Energía suministrada por el sistema PV: {Esup:.2f} kWh")
print(f"Energía no suministrada: {Eunsup:.2f} kWh")
print(f"Energía solar no utilizada: {Eunused:.2f} kWh")


In [ ]:
demand=(k['W (kW)'].loc[k.shape[0]-1]*1.25)*1e3
time=4
Is=np.mean([737,763,748,702])*4
A=3000
Ps=(Is*A) # Convertir a kW
#time=time*3600
Ps
import numpy as np
from scipy.integrate import quad

# Definición de la función a integrar
def integrand(x):
    return Ps*x

# Límites de integración

# Realizamos la integración utilizando quad
Es, error_estimado1 = quad(integrand, 0, time)


Pv=Ps*0.1 #Potencia de la planta fotovoltaica
Pv
# Definición de la función a integrar
def integrand(x):
    return demand*x

# Límites de integración

# Realizamos la integración utilizando quad
El, error_estimado2 = quad(integrand, 0, time)



#Supplied energy
if Pv > demand:
    Esup=demand*time
else:
    Esup=Pv*time

Esup
#Unsuplied energy
if Pv < demand:
    Eunsup=(demand-Pv)*time
else:
    Eunsup=0
Eunsup
# Unused solar energy
if Pv > demand:
    Eunused=(Pv-demand)*time
else:
    Eunused=0
Eunused
print(f"Energy supplied by the PV system: {Esup:.2f} kWh")
print(f"Energy unsupplied by the PV system: {Eunsup:.2f} kWh")
print(f"Unused solar energy: {Eunused:.2f} kWh")
print(f"Total energy demand: {El:.2f} kWh")
print(f"Total energy supplied by the PV system: {Pv:.2f} kWh")


In [ ]:
# Consumption of the plant
El=demand*time
El

In [ ]:
Power=k['W (kW)'].loc[k.shape[0]-1]
time=4 #horas
Energy=Power*time
#No es necesaria batería de respaldo por la zona
Energy

In [ ]:
k['W (kW)'].sum()

In [ ]:
k['P (bar)'][k.shape[0]-1]

In [ ]:
import matplotlib.pyplot as plt

# Imprime todos los estilos disponibles en Matplotlib
print(plt.style.available)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Se asume que df_resultados ya está definido y contiene las columnas:
# 'P (bar)' -> variable independiente (x)
# 'price'   -> variable dependiente (y)
#
# Si deseas probar con datos simulados, puedes descomentar el siguiente bloque:
# df_resultados = pd.DataFrame({
#     "P (bar)": np.linspace(10, 100, 20),
#     "price": 2.5 * np.linspace(10, 100, 20)**1.2 + np.random.normal(0, 20, 20)
# })

# Extraer los datos
x = df_resultados['P (bar)'].values
y = df_resultados['price'].values

# Definir el modelo: f(x) = a * x^n
def model_func(x, a, n):
    return a * x**2+n

# Estimar valores iniciales para los parámetros (p0) es muy importante en la optimización no lineal
p0 = [1.0, 1.0]  # a = 1, n = 1, por ejemplo

# Ajuste del modelo usando curve_fit
# Usamos el método 'trf' para que se puedan pasar parámetros de tolerancia (xtol) y max_nfev
popt, pcov = curve_fit(
    model_func, 
    x, 
    y, 
    p0=p0, 
    method='trf', 
    xtol=1e-3,         # Tolerancia en los cambios en la solución 
    max_nfev=10000      # Número máximo de evaluaciones de la función
)

# Los parámetros optimizados se almacenan en popt
a_fit, n_fit = popt

print("Parámetros ajustados:")
print("a =", a_fit)
print("n =", n_fit)

# Crear puntos para graficar el modelo ajustado en el rango de los datos
x_fit = np.linspace(x.min(), x.max(), 100)
y_fit = model_func(x_fit, a_fit, n_fit)

# Graficar los datos originales y la curva ajustada
plt.figure(figsize=(8, 6))
plt.scatter(x, y, label='Datos originales', color='blue')
plt.plot(x_fit, y_fit, label=f'Modelo ajustado: {a_fit:.3f}*x**{n_fit:.3f}', color='red', linewidth=2)
plt.xlabel("P (bar)", fontsize=14)
plt.ylabel("Price", fontsize=14)
plt.title("Ajuste del modelo: Price vs. P (bar)", fontsize=16)
plt.legend(fontsize=12)
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Supongamos que df_resultados ya está definido y contiene las columnas:
# 'P (bar)' para la presión y 'price' para el costo.
# Ejemplo de datos simulados (comenta o elimina estas líneas al trabajar con tus datos reales)
# df_resultados = pd.DataFrame({
#     "P (bar)": np.linspace(10, 100, 20),
#     "price": 0.05 * np.linspace(10, 100, 20)**2 - 2 * np.linspace(10, 100, 20) + 200 + np.random.normal(0, 20, 20)
# })

# Extraer datos de las columnas de interés
x = df_resultados['P (bar)'].values
y = df_resultados['price'].values

# Ajuste de un polinomio de grado 2 (parábola) a los datos
coeffs = np.polyfit(x, y, 2)  # coeffs = [a, b, c]
fitted_poly = np.poly1d(coeffs)

print("Coeficientes ajustados:")
print("a =", coeffs[0])
print("b =", coeffs[1])
print("c =", coeffs[2])

# Crear puntos para graficar la función ajustada
x_fit = np.linspace(x.min(), x.max(), 100)
y_fit = fitted_poly(x_fit)

# Graficar los datos originales y la curva ajustada
plt.figure(figsize=(8, 6))
plt.scatter(x, y, label='Datos originales', color='blue')
plt.plot(x_fit, y_fit, label='Modelo parabólico', color='red', linewidth=2)
plt.xlabel("P (bar)", fontsize=14)
plt.ylabel("Price", fontsize=14)
plt.title("Ajuste de modelo parabólico: Price vs. P (bar)", fontsize=16)
plt.legend(fontsize=12)
plt.grid(True)
plt.show()


In [ ]:
stages,temperature,Qk,Wk,waterr,price=[],[],[],[],[],[]
for i in range(len(Pes)):
    P=Pes[i]
    Po=31
    P_diff=P-Po
    P1=Po
    P2=P
    N_etapas=np.ceil((np.log(P2)/np.log(P1))/np.log(1.8))
    stages.append(N_etapas)
    P_etapa=P_diff/N_etapas
    Po=Po
    Pf=P
    P_acum=[Po]
    Temps=[25+273.15]
    Tref=Tref
    m=m
    eta=0.75
    Qs,Ts,m_H2O,comp,power,Ps=[],[],[],[],[],[]
    for i in range(int(N_etapas)):
        P1=sum(P_acum)
        print(P1)
        P_acum.append(P_etapa)
        P2=sum(P_acum)
        #Ti=(sum(P_acum)*V2*2)/(m*Z*8.314)
        #print('T',Ti)
        # Valor inicial para la incógnita x (en este caso, es un vector de una dimensión)
        x0 = [1.0]
        sol = root(trabajo_reversible_IT, x0=500, args=(m, P1, P2, eta,298.15), method='hybr')
        if sol.success:
            print(sol.message)
        else:
            print(sol.message)

        resultado = trabajo_reversible(m, P1, P2, eta, sol.x[0]-273.15,298.15)
        print(f"W: {resultado:.20f} kW")
        # Se utiliza scipy.optimize.root con método 'hybr'
        solucion = root(balance, x0, args=(sol.x[0],Tref, m), method='hybr')

        if solucion.success:
            x_encontrado = solucion.x[0]
            print(f"Solución encontrada: x = {x_encontrado:.6f}")
            Temps.append(25+273.15)
            Qs.append(float(-e))
            Ts.append((25+273.15))
            m_H2O.append(float(x_encontrado))
            comp.append('No. '+str(i+1))
            CP=19.67099783 + 0.069681519*T - 0.000200098*T**2 + 2.89493E-07*T**3 - 2.22475E-10*T**4 + 8.81466E-14*T**5 - 1.42043E-17*T**6
            CP=CP/2*1000
            power.append(resultado*1e3)
            Ps.append(P2)
        else:
            print("No se encontró solución:", solucion.message)
    # Crear un DataFrame con los datos
    data = {
        "Compressor": comp,
        "T (K)": Ts,
        "P (bar)": Ps,  # Convertir de Pa a bar
        "Cooling H2O (kg/s)": m_H2O,
        "Q (kW)": np.array(Qs),
        #"Q (kW) Aspen":np.array([-1645.66707374,-775.158866276,-514.144266522]),
        "W (kW)": np.array(power)
        #"W (kW) Aspen":np.array([2211.5522, 1052.84936, 706.504967])
        
    }

    df_compresores = pd.DataFrame(data)

    #df_compresores['Error %']=abs(((df_compresores['Q (kW) Aspen']-df_compresores['Q (kW) Python'])/df_compresores['Q (kW) Aspen']*100 + (df_compresores['W (kW) Aspen']-df_compresores['W (kW) Python'])/df_compresores['W (kW) Aspen']*100)/2)
    df_compresores.loc[df_compresores.shape[0]] = ["Total", Temps[-1], Ps[-1], sum(df_compresores['Cooling H2O (kg/s)']), sum(df_compresores['Q (kW)']), sum(df_compresores['W (kW)'])]
    calculator = PriceCalculator(df_compresores,N_etapas,CS)
    
    
    temperature.append(df_compresores.loc[df_compresores.shape[0]-1]['T (K)'])
    Qk.append(df_compresores.loc[df_compresores.shape[0]-1]['Q (kW)'])
    Wk.append(df_compresores.loc[df_compresores.shape[0]-1]['W (kW)'])
    waterr.append(df_compresores.loc[df_compresores.shape[0]-1]['Cooling H2O (kg/s)'])
    resultadot = calcular_dimension_tanque(P=df_compresores.loc[df_compresores.shape[0]-1]['P (bar)'], m=10000, crit=crit, tipo='hemi', rho_tank=4500)
    
    calculator1 = TankCostCalculator("Type I", resultadot['masa']*1e3)
    price.append(calculator.final+calculator1.final)
    

In [ ]:
resultados=pd.DataFrame()
resultados['P (bar)']=Pes
resultados['Stages']=stages
resultados['Q (kW)']=Qk
resultados['W (kW)']=Wk
resultados['H2O (kg/s)']=waterr
resultados['Price (USD)']=price

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.ticker as ticker  # Para controlar la cantidad de ticks en el eje y

# Aplicamos el estilo clásico y actualizamos algunos parámetros estéticos
#plt.style.use('classic')
plt.rcParams.update({
    'font.size': 12,             # Tamaño global de fuente
    'axes.labelsize': 12,        # Tamaño de las etiquetas de los ejes
    'axes.titlesize': 14,        # Tamaño de los títulos de los ejes
    'xtick.labelsize': 10,       # Tamaño de las etiquetas del eje x
    'ytick.labelsize': 10,       # Tamaño de las etiquetas del eje y
    'lines.linewidth': 1.8,      # Grosor de las líneas
    'lines.markersize': 6,       # Tamaño de los marcadores
    'grid.linestyle': '--',      # Estilo discontínuo para la rejilla
    'grid.linewidth': 0.75,      # Grosor de la rejilla
    'grid.color': '0.75',        # Color gris para la rejilla
})

# Se asume que las variables Pes, stages, Qk, Wk, waterr y price ya están definidas
resultados = pd.DataFrame({
    'P (bar)': Pes,
    'Stages': stages,
    'Q (kW)': Qk,
    'W (kW)': Wk,
    'H2O (kg/s)': waterr,
    'Price (USD)': price
})

# Configuración de la figura con 3 filas y 2 columnas (sin sharex para mostrar todos los valores en X)
fig, axs = plt.subplots(3, 2, figsize=(12, 12))
fig.suptitle('Variación de las variables en función de la presión', fontsize=16, fontweight='bold')

# Panel (a): Stages vs. Presión
axs[0, 0].plot(resultados['P (bar)'], resultados['Stages'],
               marker='o', linestyle='-', color='black')
axs[0, 0].set_ylabel('Stages')
axs[0, 0].set_title('(a) Etapas')
axs[0, 0].grid(True)
axs[0, 0].set_axisbelow(True)
axs[0, 0].set_xlabel('Presión (bar)')  # Se muestran los valores del eje de la presión

# Panel (b): Q (kW) vs. Presión
axs[0, 1].plot(resultados['P (bar)'], resultados['Q (kW)'],
               marker='s', linestyle='-', color='blue')
axs[0, 1].set_ylabel('Q (kW)')
axs[0, 1].set_title('(b) Transferencia de Calor')
axs[0, 1].grid(True)
axs[0, 1].set_axisbelow(True)
axs[0, 1].set_xlabel('Presión (bar)')

# Panel (c): W (kW) vs. Presión
axs[1, 0].plot(resultados['P (bar)'], resultados['W (kW)'],
               marker='^', linestyle='-', color='red')
axs[1, 0].set_ylabel('W (kW)')
axs[1, 0].set_title('(c) Trabajo')
axs[1, 0].grid(True)
axs[1, 0].set_axisbelow(True)
axs[1, 0].set_xlabel('Presión (bar)')

# Panel (d): H2O (kg/s) vs. Presión
axs[1, 1].plot(resultados['P (bar)'], resultados['H2O (kg/s)'],
               marker='d', linestyle='-', color='green')
axs[1, 1].set_ylabel('H2O (kg/s)')
axs[1, 1].set_title('(d) Caudal de Agua')
axs[1, 1].grid(True)
axs[1, 1].set_axisbelow(True)
axs[1, 1].set_xlabel('Presión (bar)')

# Panel (e): Price (USD) vs. Presión
axs[2, 0].plot(resultados['P (bar)'], resultados['Price (USD)'],
               marker='*', linestyle='-', color='purple')
axs[2, 0].set_ylabel('Price (USD)')
axs[2, 0].set_title('(e) Precio')
axs[2, 0].grid(True)
axs[2, 0].set_axisbelow(True)
axs[2, 0].set_xlabel('Presión (bar)')
# Reducir el número de ticks en el eje y a un máximo de 5 para el gráfico de Price
axs[2, 0].yaxis.set_major_locator(ticker.MaxNLocator(5))

# Ocultamos el panel vacío (última celda de la cuadrícula)
axs[2, 1].axis('off')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:


P=172
Po=31
P_diff=P-Po
P1=Po
P2=P
N_etapas=np.ceil((np.log(P2)/np.log(P1))/np.log(1.8))
N_etapas
P_etapa=P_diff/N_etapas
Po=Po
Pf=P
P_acum=[Po]
Temps=[25+273.15]
Tref=Tref
m=m
eta=0.75
Qs,Ts,m_H2O,comp,power,Ps=[],[],[],[],[],[]
for i in range(int(N_etapas)):
    P1=sum(P_acum)
    print(P1)
    P_acum.append(P_etapa)
    P2=sum(P_acum)
    #Ti=(sum(P_acum)*V2*2)/(m*Z*8.314)
    #print('T',Ti)
    # Valor inicial para la incógnita x (en este caso, es un vector de una dimensión)
    x0 = [1.0]
    sol = root(trabajo_reversible_IT, x0=500, args=(m, P1, P2, eta,298.15), method='hybr')
    if sol.success:
        print(sol.message)
    else:
        print(sol.message)

    resultado = trabajo_reversible(m, P1, P2, eta, sol.x[0]-273.15,298.15)
    print(f"W: {resultado:.20f} kW")
    # Se utiliza scipy.optimize.root con método 'hybr'
    solucion = root(balance, x0, args=(sol.x[0],Tref, m), method='hybr')

    if solucion.success:
        x_encontrado = solucion.x[0]
        print(f"Solución encontrada: x = {x_encontrado:.6f}")
        Temps.append(25+273.15)
        Qs.append(float(-e))
        Ts.append((25+273.15))
        m_H2O.append(float(x_encontrado))
        comp.append('No. '+str(i+1))
        CP=19.67099783 + 0.069681519*T - 0.000200098*T**2 + 2.89493E-07*T**3 - 2.22475E-10*T**4 + 8.81466E-14*T**5 - 1.42043E-17*T**6
        CP=CP/2*1000
        power.append(resultado*1e3)
        Ps.append(P2)
    else:
        print("No se encontró solución:", solucion.message)
# Crear un DataFrame con los datos
data = {
	"Compressor": comp,
    "T (K)": Ts,
    "P (bar)": Ps,  # Convertir de Pa a bar
	"Cooling H2O (kg/s)": m_H2O,
	"Q (kW)": np.array(Qs),
    #"Q (kW) Aspen":np.array([-1645.66707374,-775.158866276,-514.144266522]),
    "W (kW)": np.array(power)
    #"W (kW) Aspen":np.array([2211.5522, 1052.84936, 706.504967])
    
}

df_compresores = pd.DataFrame(data)

#df_compresores['Error %']=abs(((df_compresores['Q (kW) Aspen']-df_compresores['Q (kW) Python'])/df_compresores['Q (kW) Aspen']*100 + (df_compresores['W (kW) Aspen']-df_compresores['W (kW) Python'])/df_compresores['W (kW) Aspen']*100)/2)
df_compresores.loc[df_compresores.shape[0]] = ["Total", Temps[-1], Ps[-1], sum(df_compresores['Cooling H2O (kg/s)']), sum(df_compresores['Q (kW)']), sum(df_compresores['W (kW)'])]
df_compresores


In [ ]:

calculator = PriceCalculator(df_compresores,N_etapas,CS)

In [ ]:
calculator.dataframe

In [ ]:
Tref

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import root, minimize

def normalize_price(price):
    """
    Convierte un valor de precio que puede venir formateado (por ejemplo, "$1,234.56")
    a un número tipo float.
    Si 'price' es un pandas.Series con un solo valor, se extrae ese valor.
    """
    # Si es una serie (con un solo valor), extraer el valor.
    if isinstance(price, pd.Series):
        price = price.iloc[0]
    
    # Si ya es numérico, lo retorna en forma float.
    if isinstance(price, (int, float)):
        return float(price)
    
    if isinstance(price, str):
        # Eliminar símbolos comunes: $, €, espacios y comas
        cleaned = price.replace("$", "").replace("€", "").replace(",", "").strip()
        try:
            return float(cleaned)
        except Exception as ex:
            raise ValueError("No se pudo normalizar el precio: '{}'".format(price))
    # De lo contrario, se intenta convertir a float
    return float(price)

def cost_function(P, m, Tref, T, CS, crit):
    """
    Función objetivo para minimizar; dada una presión P (en bar) calcula el costo total,
    el cual es la suma de:
      - El costo de compresión obtenido con PriceCalculator.
      - El costo del tanque obtenido con TankCostCalculator.
    
    Se normalizan los precios para asegurar que sean float antes de realizar la suma.

    Parámetros:
      - P   : Presión final (en bar). Es la variable independiente a optimizar.
      - m   : Parámetro relacionado a la masa o cantidad a comprimir.
      - Tref: Temperatura de referencia (en Kelvin).
      - T   : Temperatura usada en algunos cálculos (ej. en la ecuación de Cp).
      - CS  : Constante o parámetro requerido por PriceCalculator.
      - crit: Diccionario con las propiedades críticas (keys: 'Tc', 'Pc', 'PM', 'w').
    
    Devuelve:
      total_cost: Suma de los costos (float) de compresión y de tanque.
    """

    # Fijamos la presión de entrada (Po) a 31 bar.
    Po = 31  
    P=P[0]
    if P <= Po:
        # Penalizar fuertemente si la presión final no supera la de entrada.
        return 1e12

    P_diff = P - Po
    P1 = Po
    P2 = P
    # Estimación del número de etapas. Se redondea hacia arriba.
    N_etapas= np.ceil((np.log(P2) / np.log(P1)) / np.log(1.8))
    # Incremento de presión por etapa:
    P_etapa = P_diff / N_etapas

    # Variables iniciales para las etapas
    P_acum = [Po]            # Acumulador de presión
    Temps = [25 + 273.15]    # Temperatura en Kelvin (25°C + 273.15)
    eta = 0.75               # Eficiencia
    # Inicializar listas para los resultados de cada etapa
    Qs, Ts, m_H2O, comp, power, Ps_list = [], [], [], [], [], []
    
    # Bucle sobre las etapas de compresión:
    for j in range(int(N_etapas)):
        # La presión de entrada actual es la suma de los incrementos acumulados.
        P1_current = sum(P_acum)
        # Agregar la presión de la etapa y actualizar la presión de salida.
        P_acum.append(P_etapa)
        P2_current = sum(P_acum)
        
        # Resolver la ecuación de trabajo reversible (usando una función externa)
        sol = root(trabajo_reversible_IT, x0=500, args=(m, P1_current, P2_current, eta, 298.15), method='hybr')
        if not sol.success:
            return 1e12  # Penalización si no se converge

        # Calcular el trabajo (en kW) usando la función trabajo_reversible.
        # Se utiliza sol.x[0] - 273.15 para convertir de Kelvin a °C.
        resultado = trabajo_reversible(m, P1_current, P2_current, eta, sol.x[0] - 273.15, 298.15)
        
        # Resolver la ecuación de balance (por ejemplo, para obtener el caudal de enfriamiento),
        # usando root con un valor inicial.
        x0_vec = [1.0]
        solucion = root(balance, x0_vec, args=(sol.x[0], Tref, m), method='hybr')
        if not solucion.success:
            return 1e12
        x_encontrado = solucion.x[0]
        
        # Se asume un valor placeholder para "e" (ya que no está definido, se utiliza 0)
        e_value = 0  
        
        # Acumulación de datos para la etapa.
        Temps.append(25 + 273.15)
        Qs.append(float(-e_value))  # Aquí se debe introducir el valor correcto para Q.
        Ts.append(25 + 273.15)
        m_H2O.append(float(x_encontrado))
        comp.append('No. ' + str(j + 1))
        # Se calcula CP con una expresión polinómica (ejemplo, sin utilizarlo directamente)
        CP = (19.67099783 
              + 0.069681519 * T 
              - 0.000200098 * T**2 
              + 2.89493E-07 * T**3 
              - 2.22475E-10 * T**4 
              + 8.81466E-14 * T**5 
              - 1.42043E-17 * T**6)
        CP = CP / 2 * 1000
        
        power.append(resultado * 1e3)  # Convertir a Watts, por ejemplo.
        Ps_list.append(P2_current)
    
    # Armado del DataFrame con información de cada etapa.
    data = {
        "Compressor": comp,
        "T (K)": Ts,
        "P (bar)": Ps_list,  # Se trabaja con P en bar
        "Cooling H2O (kg/s)": m_H2O,
        "Q (kW)": np.array(Qs),
        "W (kW)": np.array(power)
    }
    df_compresores = pd.DataFrame(data)
    # Agregar una fila "Total" que resume los valores.
    total_row = ["Total",
                 Temps[-1],
                 Ps_list[-1],
                 df_compresores["Cooling H2O (kg/s)"].sum(),
                 df_compresores["Q (kW)"].sum(),
                 df_compresores["W (kW)"].sum()]
    df_compresores.loc[df_compresores.shape[0]] = total_row

    # --- Cálculo de costos ---
    # Se obtiene el costo del proceso de compresión.
    calculator = PriceCalculator(df_compresores, N_etapas, CS)
    cost_compression = normalize_price(calculator.final)

    # Se dimensiona el tanque utilizando la función calcular_dimension_tanque.
    resultadot = calcular_dimension_tanque(
        P=sum(Ps_list),
        m=10000,
        crit=crit,
        tipo='hemi',
        rho_tank=4500
    )
    # Se calcula el costo del tanque.
    calculator1 = TankCostCalculator("Type I", resultadot["masa"] * 1e3)
    cost_tank = normalize_price(calculator1.extras_df["Final Cost (USD)"])

    # Sumar ambos costos para obtener el costo total.
    total_cost = cost_compression + cost_tank
    print
    print('TOTAL:',total_cost)
    return total_cost

# --- Ejemplo de optimización usando scipy.minimize ---
if __name__ == '__main__':
    # Parámetros o variables del proceso (se deben definir según tu aplicación)
    m = 4/86400           # Ejemplo: masa o caudal relacionado
    Tref = Tref      # Temperatura de referencia en Kelvin
    T = 298.15         # Temperatura usada para ciertos cálculos (como CP)
    CS = "Carbon Steel"           # Constante para PriceCalculator (ejemplo)
    crit = crit
    
    # Definir límites para la presión óptima (en bar). Po es 31 en el código, así que se debe estar por encima.
    P_min = 50  
    P_max = 1000
    x0 = [900]  # Valor inicial en bar
    
    # Ejecutar la optimización
    res = minimize(cost_function, x0, args=(m, Tref, T, CS, crit),
                   bounds=[(P_min, P_max)])
    
    if res.success:
        print("Presión óptima encontrada (bar):", res.x[0])
        print("Costo total mínimo:", res.fun)
    else:
        print("La optimización falló:", res.message)
